In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time
import os
import sys
sys.path.append(os.path.abspath(".."))
import packages.Preprocesamiento as ppr

# Diccionario de hoteles con URLs
hoteles = {
    "Vitoria": 'https://www.booking.com/reviews/es/hotel/libere-vitoria-centro.es.html?aid=356980&label=gog235jc-1DCA0oRkIVbGliZXJlLXZpdG9yaWEtY2VudHJvSDNYA2hGiAEBmAEKuAEXyAEM2AED6AEBiAIBqAIDuALSzuu6BsACAdICJDBiNGMxOWY2LWVlOGMtNDU3ZS05ZDZlLTVmYzM2NDNjZTM5M9gCBOACAQ&sid=b89a34b9ace8403db8dbb71ddde917f9&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&page=1&r_lang=es&rows=75&',
    "ValenciaAbastos":    'https://www.booking.com/reviews/es/hotel/apartamentos-libere-valencia-abastos.es.html',
    "PamplonaYamaguchi":    'https://www.booking.com/reviews/es/hotel/apartamentos-libere-pamplona-yamaguchi.es.html',
    "ValenciaJardinBotanico":    'https://www.booking.com/reviews/es/hotel/apartamentos-libere-valencia-jardin-botanico.es.html',
    "MadridPalacioReal":    'https://www.booking.com/reviews/es/hotel/libere-madrid-palacio-real.es.html',
    "MalagaTeatroRomano":    'https://www.booking.com/reviews/es/hotel/apartamentosliberemalagateatroromano.es.html',
    "GranadaCatedral":    'https://www.booking.com/reviews/es/hotel/apartamentos-libere-granada-catedral.es.html',
    "MalagaLaMerced":    'https://www.booking.com/reviews/es/hotel/libere-malaga-la-merced.es.html',
    "CordobaPatio":    'https://www.booking.com/reviews/es/hotel/libere-cordoba-patio-santa-marta.es.html?aid=356980&label=gog235jc-1DCA0oRkIgbGliZXJlLWNvcmRvYmEtcGF0aW8tc2FudGEtbWFydGFIM1gDaEaIAQGYAQq4ARfIAQzYAQPoAQGIAgGoAgO4AruB8LoGwAIB0gIkZWRiMTQzODktYmYyZi00NGJkLTg4MDYtNzBlYjc1YmFiZTM12AIE4AIB&sid=b89a34b9ace8403db8dbb71ddde917f9&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&page=1&r_lang=es&rows=75&',
    "Donosti":   'https://www.booking.com/reviews/es/hotel/koisi-hostel.es.html?aid=356980&label=gog235jc-1DCA0oRkIMa29pc2ktaG9zdGVsSDNYA2hGiAEBmAEKuAEXyAEM2AED6AEBiAIBqAIDuAKIzuu6BsACAdICJDBkYmJjNGJmLTBiYzYtNDUzNS1hYjM2LWU5N2RlMTFkZDU2N9gCBOACAQ&sid=b89a34b9ace8403db8dbb71ddde917f9&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&page=1&r_lang=es&rows=75&',
    "BilbaoMuseo":  'https://www.booking.com/reviews/es/hotel/apartamentos-libere-bilbao-guggenheim.es.html',
    "BilbaoLaVieja":   'https://www.booking.com/reviews/es/hotel/apartamentos-libere-bilbao-la-vieja.es.html',
    }

headers = {"User-Agent": "Mozilla/5.0"}
todos_comentarios = []


def scrape_hotel(hotel, url_base):
    pagina = 1
    hotel_comentarios = []

    
    if "page=" not in url_base:
        url_base += "&page=1" if "?" in url_base else "?page=1"

    while True:
        url = url_base.replace("page=1", f"page={pagina}")
        print(f"[{hotel}] Obteniendo los datos de la página {pagina}")
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")

        reviews = soup.select("li.review_item.clearfix")
        titulos = soup.select("div.review_item_header_content span[itemprop='name']")

        if not reviews or len(titulos) == 0:
            print(f"[{hotel}] No hay más reseñas.")
            break

        for review in reviews:
            # Autor
            autor_tag = review.select_one("p.reviewer_name span")
            autor = autor_tag.get_text(strip=True) if autor_tag else "Autor no encontrado"

            # Fecha
            fecha_tag = review.select_one("p.review_item_date")
            fecha = fecha_tag.get_text(strip=True).replace("Escrito en", "").strip() if fecha_tag else "Fecha no encontrada"

            # Título
            titulo_tag = review.select_one("div.review_item_header_content span[itemprop='name']")
            titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Titulo no encontrado"

            # Comentarios
            comentario_neg_tag = review.select_one("p[class*='neg'] span[itemprop='reviewBody']")
            comentario_negativo = comentario_neg_tag.get_text(strip=True) if comentario_neg_tag else 'Comentario negativo no encontrado'

            comentario_pos_tag = review.select_one("p[class*='pos'] span[itemprop='reviewBody']")
            comentario_positivo = comentario_pos_tag.get_text(strip=True) if comentario_pos_tag else 'Comentario positivo no encontrado'

            # Nota
            nota_tag = review.select_one("span.review-score-badge")
            nota = nota_tag.get_text(strip=True).replace(",", ".") if nota_tag else 'Nota no encontrada'

            # Procedencia
            proc_tag = review.select_one("div.review_item_reviewer span[itemprop='nationality']")
            procedencia = proc_tag.get_text(strip=True) if proc_tag else 'Procedencia no encontrada'

            # Tags del usuario
            tags = []
            for li in review.select("ul.review_item_info_tags li.review_info_tag"):
                texto = li.get_text(strip=True).replace("•", "").strip()
                if texto:
                    tags.append(texto)

            hotel_comentarios.append({
                "hotel": hotel,
                "autor": autor,
                "pais del autor": procedencia,
                "fecha": fecha,
                "titulo": titulo,
                "comentario negativo": comentario_negativo,
                "comentario positivo": comentario_positivo,
                "nota": nota,
                "tags": ", ".join(tags)
            })

        pagina += 1
        time.sleep(1)  

    return hotel_comentarios

# Paralelizar 
with ThreadPoolExecutor(max_workers=8) as executor:
    resultados = executor.map(lambda x: scrape_hotel(x[0], x[1]), hoteles.items())

for hotel_res in resultados:
    todos_comentarios.extend(hotel_res)

# Guardat fatos en un objeto
ppr.guardar_objeto(obj=todos_comentarios, nombre='comentarios_hoteles', destino='trf')
print("Scraping terminado. El objeto generado es: comentarios_hoteles")

[Vitoria] Obteniendo los datos de la página 1
[ValenciaAbastos] Obteniendo los datos de la página 1
[PamplonaYamaguchi] Obteniendo los datos de la página 1
[ValenciaJardinBotanico] Obteniendo los datos de la página 1
[MadridPalacioReal] Obteniendo los datos de la página 1
[MalagaTeatroRomano] Obteniendo los datos de la página 1
[GranadaCatedral] Obteniendo los datos de la página 1
[MalagaLaMerced] Obteniendo los datos de la página 1
[PamplonaYamaguchi] Obteniendo los datos de la página 2
[ValenciaAbastos] Obteniendo los datos de la página 2
[Vitoria] Obteniendo los datos de la página 2
[MadridPalacioReal] Obteniendo los datos de la página 2
[ValenciaJardinBotanico] Obteniendo los datos de la página 2
[MalagaLaMerced] Obteniendo los datos de la página 2
[MalagaTeatroRomano] Obteniendo los datos de la página 2
[GranadaCatedral] Obteniendo los datos de la página 2
[PamplonaYamaguchi] Obteniendo los datos de la página 3[ValenciaAbastos] Obteniendo los datos de la página 3

[Vitoria] Obteni